RAG Using Azure AI Agent Service & Semantic Kernel

In [1]:
from dotenv import load_dotenv
import os
from azure.identity.aio import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential
from azure.identity import ClientSecretCredential
from azure.ai.projects import AIProjectClient
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from azure.ai.agents.models import FileSearchTool, VectorStore, FilePurpose 

Document Upload, Vector Store Creation

In [2]:
async def upload_file_create_vector_store(client) -> tuple[str, VectorStore]:
    file = await client.agents.files.upload_and_poll(
        file="ContosoTravel.md",
        purpose=FilePurpose.AGENTS
    )
    vector_store = await client.agents.create_vector_store_and_poll(
        file_ids=[file.id], name="my_vectorstore"
    )
    return file, vector_store

Agent

In [3]:

AGENT_NAME = "RAGAgent",
AGENT_INSTRUCTIONS = """
You are an AI assistant designed to answer user questions using only the informations retrieved from the provided documents
- If a user's question can not be answered using the retrieved context, you must clearly respond:
"I am sorry, but the uploaded document does not contain the necessary information to answer that question"
- Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
- Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
- If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
- For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.
You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
"""
async def create_agent(client, vector_store_id):
    file_search = FileSearchTool(vector_store_ids=vector_store_id)
    agent_definition = await client.agents.create_agent(
        model="gpt-4o-mini",
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
        tools=file_search.definitions,
        tool_resources=file_search.resources,
    )
    return AzureAIAgent(client=client, definition=agent_definition)

Main

In [4]:
load_dotenv()
project_endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]
tenant_id = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")

cred = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret,
)
# cred = InteractiveBrowserCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))
# client = AIProjectClient(endpoint=project_endpoint, credential=cred)

In [ ]:
async def main():
    async with (
        # DefaultAzureCredential() as cred,
        AzureAIAgent.create_client(endpoint=project_endpoint, credential=cred) as client,
    ):
        file, vector_store = await upload_file_create_vector_store(client)
        agent = await create_agent(client, vector_store.id)

        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",
            "What is a Neural Network?"
        ]

        thread = None
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()